In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder

C:\Users\Keshav\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Keshav\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Keshav\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
flight_price = pd.read_excel(r'./Flight_Ticket_Participant_Datasets/Data_Train.xlsx')
flight_price.head(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [3]:
flight_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


# Check NULL value

In [4]:
flight_price.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

### drop no data

In [5]:
flight_price = flight_price.dropna()

# unique value counts

In [6]:
flight_price['Airline'].value_counts()

Jet Airways                          3849
IndiGo                               2053
Air India                            1751
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: Airline, dtype: int64

In [7]:
flight_price['Total_Stops'].value_counts()

1 stop      5625
non-stop    3491
2 stops     1520
3 stops       45
4 stops        1
Name: Total_Stops, dtype: int64

In [8]:
flight_price['Additional_Info'].value_counts()

No info                         8344
In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
Red-eye flight                     1
1 Short layover                    1
2 Long layover                     1
Name: Additional_Info, dtype: int64

In [9]:
flight_price['Duration'].value_counts()

2h 50m     550
1h 30m     386
2h 45m     337
2h 55m     337
2h 35m     329
          ... 
27h 30m      1
36h 25m      1
31h 50m      1
41h 20m      1
29h 40m      1
Name: Duration, Length: 368, dtype: int64

# Data Transformation / Feature Creation

In [10]:
flight_price["Journey_day"] = pd.to_datetime(flight_price.Date_of_Journey, format="%d/%m/%Y").dt.day
flight_price["Journey_month"] = pd.to_datetime(flight_price["Date_of_Journey"], format = "%d/%m/%Y").dt.month
flight_price["Journey_year"] = pd.to_datetime(flight_price["Date_of_Journey"], format = "%d/%m/%Y").dt.year



In [11]:
# Extracting Hours
flight_price["Dep_hour"] = pd.to_datetime(flight_price["Dep_Time"]).dt.hour

# Extracting Minutes
flight_price["Dep_min"] = pd.to_datetime(flight_price["Dep_Time"]).dt.minute



In [12]:
duration=list(flight_price['Duration'])
Duration_hours = list()
Duration_minute = list()

for i in range(len(duration)):
    if len(duration[i].split(' '))==2:
        Duration_hours.append(duration[i].split(' ')[0])
        Duration_minute.append(duration[i].split(' ')[1])
    else:
        if 'h' in duration[i]: # Check if duration contains only hour
            Duration_hours.append(duration[i].split(' ')[0])
            Duration_minute.append('0m')   
        else:
            Duration_hours.append('0h')
            Duration_minute.append(duration[i].split(' ')[0])


In [13]:
flight_price["Duration_hours"] = Duration_hours
flight_price["Duration_minute"] = Duration_minute

In [14]:
flight_price['Duration_hours'].value_counts()

2h     2402
1h      621
3h      501
7h      487
5h      481
9h      445
12h     428
8h      424
13h     407
11h     365
10h     355
6h      340
14h     337
15h     268
23h     264
26h     241
16h     234
4h      222
22h     218
24h     197
21h     196
25h     186
27h     179
20h     162
18h     141
19h     134
17h     129
28h      94
29h      65
30h      49
38h      34
37h      17
33h      13
32h       9
34h       8
36h       7
35h       7
31h       6
39h       2
42h       2
47h       2
41h       1
40h       1
0h        1
Name: Duration_hours, dtype: int64

In [15]:
flight_price['Duration_minute'].value_counts()

30m    1446
0m     1031
20m     997
50m     972
35m     939
55m     910
15m     903
45m     896
25m     803
40m     636
5m      624
10m     525
Name: Duration_minute, dtype: int64

In [16]:
flight_price['Duration_hours'] = flight_price['Duration_hours'].str.replace('h', '')
flight_price['Duration_minute'] = flight_price['Duration_minute'].str.replace('m', '')


In [17]:
flight_price.drop(["Date_of_Journey","Dep_Time","Duration"], axis = 1, inplace = True)

### drop not require feature

In [18]:
### Route will not decide the flight price
### Arrival time also not decide the flight price
flight_price.drop(["Arrival_Time","Route"], axis = 1, inplace = True)

In [19]:
flight_price

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Journey_day,Journey_month,Journey_year,Dep_hour,Dep_min,Duration_hours,Duration_minute
0,IndiGo,Banglore,New Delhi,non-stop,No info,3897,24,3,2019,22,20,2,50
1,Air India,Kolkata,Banglore,2 stops,No info,7662,1,5,2019,5,50,7,25
2,Jet Airways,Delhi,Cochin,2 stops,No info,13882,9,6,2019,9,25,19,0
3,IndiGo,Kolkata,Banglore,1 stop,No info,6218,12,5,2019,18,5,5,25
4,IndiGo,Banglore,New Delhi,1 stop,No info,13302,1,3,2019,16,50,4,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,non-stop,No info,4107,9,4,2019,19,55,2,30
10679,Air India,Kolkata,Banglore,non-stop,No info,4145,27,4,2019,20,45,2,35
10680,Jet Airways,Banglore,Delhi,non-stop,No info,7229,27,4,2019,8,20,3,0
10681,Vistara,Banglore,New Delhi,non-stop,No info,12648,1,3,2019,11,30,2,40


# one hot encoding

In [20]:
one_hot_encoding_features = ["Airline","Source","Destination","Total_Stops","Additional_Info"]

In [21]:
#new_flight_dataset = pd.get_dummies(flight_price, columns = one_hot_encoding_features)
enc = OneHotEncoder(handle_unknown='ignore',sparse =False)

enc.fit(flight_price[one_hot_encoding_features])

temp_ = pd.DataFrame(enc.transform(flight_price[one_hot_encoding_features]))

flight_price = flight_price.drop(one_hot_encoding_features,axis=1)


flight_price = pd.concat([flight_price,temp_],axis=1)
flight_price

,Price,Journey_day,Journey_month,Journey_year,Dep_hour,Dep_min,Duration_hours,Duration_minute,0,1,...,28,29,30,31,32,33,34,35,36,37
0,3897.0,24.0,3.0,2019.0,22.0,20.0,2,50,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,7662.0,1.0,5.0,2019.0,5.0,50.0,7,25,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,13882.0,9.0,6.0,2019.0,9.0,25.0,19,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6218.0,12.0,5.0,2019.0,18.0,5.0,5,25,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,13302.0,1.0,3.0,2019.0,16.0,50.0,4,45,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,4107.0,9.0,4.0,2019.0,19.0,55.0,2,30,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10679,4145.0,27.0,4.0,2019.0,20.0,45.0,2,35,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10680,7229.0,27.0,4.0,2019.0,8.0,20.0,3,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10681,12648.0,1.0,3.0,2019.0,11.0,30.0,2,40,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
flight_price = flight_price.dropna()

In [24]:
new_flight_dataset = flight_price.copy()

### Machine Learning Model

In [25]:
### dived datat into depended and indepedent features
X_train = new_flight_dataset.drop(["Price"], axis=1)
Y_train = new_flight_dataset["Price"]

# Linear Regression

In [26]:
### intitilze LR model
lr=LinearRegression()

### fit the LR model on train dataset
lr.fit(X_train,Y_train)


    
### Modle Training Performance
print("model performance on train data\n")
Y_pred_train=lr.predict(X_train)
print('r2_score for train data')

print(r2_score(Y_train,Y_pred_train))


mse = mean_squared_error(Y_train,Y_pred_train)

rmse = math.sqrt(mse)
print('rmse for train data:', rmse)






model performance on train data

r2_score for train data
0.5419696996649686
rmse for train data: 3120.952220027998


# Rando Forest Regressor

In [27]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [5,10,15],
    'n_estimators': [10,30,70,100]
}

rf=RandomForestRegressor()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,cv = 3, n_jobs = 1, verbose = 2)

grid_search.fit(X_train,Y_train)


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] bootstrap=True, max_depth=5, n_estimators=10 ....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... bootstrap=True, max_depth=5, n_estimators=10, total=   0.3s
[CV] bootstrap=True, max_depth=5, n_estimators=10 ....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ..... bootstrap=True, max_depth=5, n_estimators=10, total=   0.1s
[CV] bootstrap=True, max_depth=5, n_estimators=10 ....................
[CV] ..... bootstrap=True, max_depth=5, n_estimators=10, total=   0.1s
[CV] bootstrap=True, max_depth=5, n_estimators=30 ....................
[CV] ..... bootstrap=True, max_depth=5, n_estimators=30, total=   0.3s
[CV] bootstrap=True, max_depth=5, n_estimators=30 ....................
[CV] ..... bootstrap=True, max_depth=5, n_estimators=30, total=   0.3s
[CV] bootstrap=True, max_depth=5, n_estimators=30 ....................
[CV] ..... bootstrap=True, max_depth=5, n_estimators=30, total=   0.3s
[CV] bootstrap=True, max_depth=5, n_estimators=70 ....................
[CV] ..... bootstrap=True, max_depth=5, n_estimators=70, total=   0.8s
[CV] bootstrap=True, max_depth=5, n_estimators=70 ....................
[CV] ..... bootstrap=True, max_depth=5, n_estimators=70, total=   0.8s
[CV] bootstrap=True, max_depth=5, n_estimators=70 ....................
[CV] .

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:   43.7s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'bootstrap': [True], 'max_depth': [5, 10, 15], 'n_estimators': [10, 30, 70, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [29]:
print("best model parameter")
print(grid_search.best_estimator_)

    
### Modle Training Performance
print("model performance on train data\n")
Y_pred_train=grid_search.predict(X_train)
print('r2_score for train data')

print(r2_score(Y_train,Y_pred_train))


mse = mean_squared_error(Y_train,Y_pred_train)

rmse = math.sqrt(mse)
print('rmse for train data:', rmse)






best model parameter
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
model performance on train data

r2_score for train data
0.9475926268725905
rmse for train data: 1055.6895222540518


### best model will save as joblib

Random forest regrossor have good rsqr and low rmse lalue compare to linear regression

In [30]:
# save the model to disk
filename = 'finalized_model.joblib'
joblib.dump(grid_search, filename)

['finalized_model.joblib']

# Test Data

In [33]:
flight_price = pd.read_excel(r'./Flight_Ticket_Participant_Datasets/Test_set.xlsx')


In [34]:
flight_price["Journey_day"] = pd.to_datetime(flight_price.Date_of_Journey, format="%d/%m/%Y").dt.day
flight_price["Journey_month"] = pd.to_datetime(flight_price["Date_of_Journey"], format = "%d/%m/%Y").dt.month
flight_price["Journey_year"] = pd.to_datetime(flight_price["Date_of_Journey"], format = "%d/%m/%Y").dt.year



In [35]:
# Extracting Hours
flight_price["Dep_hour"] = pd.to_datetime(flight_price["Dep_Time"]).dt.hour

# Extracting Minutes
flight_price["Dep_min"] = pd.to_datetime(flight_price["Dep_Time"]).dt.minute



In [36]:
duration=list(flight_price['Duration'])
Duration_hours = list()
Duration_minute = list()

for i in range(len(duration)):
    if len(duration[i].split(' '))==2:
        Duration_hours.append(duration[i].split(' ')[0])
        Duration_minute.append(duration[i].split(' ')[1])
    else:
        if 'h' in duration[i]: # Check if duration contains only hour
            Duration_hours.append(duration[i].split(' ')[0])
            Duration_minute.append('0m')   
        else:
            Duration_hours.append('0h')
            Duration_minute.append(duration[i].split(' ')[0])


In [37]:
flight_price["Duration_hours"] = Duration_hours
flight_price["Duration_minute"] = Duration_minute

In [38]:
flight_price['Duration_hours'] = flight_price['Duration_hours'].str.replace('h', '')
flight_price['Duration_minute'] = flight_price['Duration_minute'].str.replace('m', '')


In [39]:
flight_price.drop(["Date_of_Journey","Dep_Time","Duration"], axis = 1, inplace = True)

In [40]:
### Route will not decide the flight price
### Arrival time also not decide the flight price
flight_price.drop(["Arrival_Time","Route"], axis = 1, inplace = True)

In [41]:
one_hot_encoding_features = ["Airline","Source","Destination","Total_Stops","Additional_Info"]

In [42]:
temp_ = pd.DataFrame(enc.transform(flight_price[one_hot_encoding_features]))

flight_price = flight_price.drop(one_hot_encoding_features,axis=1)


flight_price = pd.concat([flight_price,temp_],axis=1)
flight_price

,Journey_day,Journey_month,Journey_year,Dep_hour,Dep_min,Duration_hours,Duration_minute,0,1,2,...,28,29,30,31,32,33,34,35,36,37
0,6,6,2019,17,30,10,55,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,12,5,2019,6,20,4,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,21,5,2019,19,15,23,45,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,21,5,2019,8,0,13,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,24,6,2019,23,55,2,50,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,6,6,2019,20,30,23,55,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2667,27,3,2019,14,20,2,35,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2668,6,3,2019,21,50,6,35,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2669,6,3,2019,4,0,15,15,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [43]:
flight_price = flight_price.dropna()

In [44]:
new_flight_dataset = flight_price.copy()

In [46]:
X_test = new_flight_dataset.copy()

In [50]:
Y_pred_test=grid_search.predict(X_test)
Y_pred_test=pd.DataFrame(Y_pred_test,columns=['Price'])

Y_pred_test.to_csv("output.csv")
print(Y_pred_test)

             Price
0     14436.327273
1      4226.000000
2     12891.937552
3     10960.308149
4      3889.942587
...            ...
2666   9606.190911
2667   4961.727431
2668  15786.186364
2669  13138.341648
2670   7734.805674

[2671 rows x 1 columns]
